In [5]:
import pandas as pd
df_movies = pd.read_csv('data/movies.csv')
df_ratings1 = pd.read_csv("data/ratings1.csv")
df_ratings2 = pd.read_csv("data/ratings2.csv")
df_mdates = pd.read_csv("data/dates.csv")

# Подсчет количества уникальных значений в столбце 'title'
unique_movies_count = df_movies['title'].nunique()

# Подсчет количества уникальных пользователей в таблице ratings1
unique_users1_count = df_ratings1['userId'].nunique()

# В каком году было выставлено больше всего оценок?
df_mdates['date'] = pd.to_datetime(df_mdates["date"])
# Извлекаем год из столбца 'date'
df_mdates['year'] = df_mdates['date'].dt.year
# Подсчет количества оценок для каждого года
years_counts = df_mdates['year'].value_counts()
# Нахождение года с наибольшим количеством оценок
max_year = years_counts.idxmax()
max_year_count = years_counts.max()

print(f'Количество уникальных фильмов: {unique_movies_count}')
print(f'Количество уникальных пользователей: {unique_users1_count}')
print(f"Год с наибольшим количеством оценок: {max_year} (Количество оценок: {max_year_count})")


Количество уникальных фильмов: 9737
Количество уникальных пользователей: 274
Год с наибольшим количеством оценок: 2000 (Количество оценок: 10061)


In [17]:
# склеим две таблицы ratings1 и ratings2
ratings = pd.concat(
    [df_ratings1, df_ratings2],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [18]:
# узнаем количество строк в таблицах ratings и dates, ведь нам предстоит вертикально склеить их между собой

print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', df_mdates.shape[0])
print(ratings.shape[0] == df_mdates.shape[0])

# находим дублирующие строки
display(df_ratings1.tail(1))
display(df_ratings2.head(1))

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


In [19]:
# убираем дубликаты
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


In [20]:
# можем теперь добавить к нашей таблице с оценками даты их выставления
ratings_dates = pd.concat([ratings, df_mdates], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date,year
100829,610,164179,5.0,2017-05-03 21:07:11,2017
100830,610,166528,4.0,2017-05-04 06:29:25,2017
100831,610,166534,4.0,2017-05-03 21:53:22,2017
100832,610,168248,5.0,2017-05-03 22:21:31,2017
100833,610,168250,5.0,2017-05-08 19:50:47,2017
100834,610,168252,5.0,2017-05-03 21:19:12,2017
100835,610,170875,3.0,2017-05-03 21:20:15,2017


In [22]:
# Пробуем влоб объединить две таблицы
joined_false = ratings_dates.join(
    df_movies,
    rsuffix='_right',
    how='left'
)
display(joined_false)

,userId,movieId,rating,date,year,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,2000,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,2000,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,2000,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,2017,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,2017,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,2017,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,2017,NaN,NaN,NaN


In [23]:
# Мы не получили соответствие фильмов и их рейтингов, пробуем снова используя ключевой столбец в "правой" таблице
# в качестве индекса с помощью метода set_index()
joined = ratings_dates.join(
    df_movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

,userId,movieId,rating,date,year,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,2000,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,2000,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,2000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [26]:
# Пробуем объединение с более предпочтительным методом merge()
merged = ratings_dates.merge(
    df_movies,
    on='movieId',
    how='left'
)
display(merged.head())
# Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей таблице merged:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

,userId,movieId,rating,date,year,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,2000,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,2000,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,2000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True
